# Track 2: SIR Model

Lorem Ipsum

## Modelling epidemic spread of disease with the SIR model

Lorem Ipsum

In [2]:
#first we install the package (in case it is not done yet) and import the Simulation class
%pip install -e .
from pygmodw23.sims import Simulation

Obtaining file:///home/david/Desktop/PygameModelling23
  Attempting uninstall: PyGame-Modelling-Workshop-2023
    Found existing installation: PyGame-Modelling-Workshop-2023 1.0.0
    Uninstalling PyGame-Modelling-Workshop-2023-1.0.0:
      Successfully uninstalled PyGame-Modelling-Workshop-2023-1.0.0
  Running setup.py develop for PyGame-Modelling-Workshop-2023
Note: you may need to restart the kernel to use updated packages.
pygame 2.4.0 (SDL 2.26.4, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [13]:
# Simulation Parameters
# simulation time
T = 1000
# number of agents
N = 100
# path trail lengths
memory_length = 0
# arena
width = height = 500
window_pad = 50

# Create a simulation instance
test_simulation = Simulation(N=N, T=T, agent_type="SIR-brownian-selfpropelled", framerate=30,
                            width=width, height=height, window_pad=window_pad)

# Change Agent Parameters
# a.) type of noise ("normal" or "uniform")
# b.) noise parameters (min, max) or (mean, std) of movement noise
# c.) particle speed
for agent in test_simulation.agents:
    agent.noise_type = "uniform"  # or "normal"
    agent.noise_params = (-0.5, 0.5)  # (min, max) or (mean, std)
    agent.v_max = 2             # constant maximum absolute velocity
    agent.infection_prob = 0.005
    agent.recovery_prob = 0.001
    agent.death_prob = 0.005
    agent.infection_radius = 30


# Initially infecting some agents
num_infected_agents = 3
for agent in test_simulation.agents:
    if agent.id < num_infected_agents:
        agent.state = "I"
    else:
        break
        
    
# showing colorcoded trails
test_simulation.memory_length = memory_length


# Start the simulation
test_simulation.start()

Running simulation start method!
Starting main simulation loop!
2023-06-05_21-39-16.728998 Total simulation time:  33.248391
